In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("assignment2").getOrCreate()

### Load data and create a Spark data frame

In [4]:
df = spark.read.option("delimiter", ";").csv("Project 1_dataset_bank-full.csv", inferSchema=True, header=True, )

In [5]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [6]:
df.show(1)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
only showing top 1 row



### Give marketing success rate (No. of people subscribed / total no. of entries)

In [7]:
df.filter(df['y'] == 'yes').count()/df.count() * 100

11.698480458295547

### Give marketing failure rate

In [8]:
df.filter(df['y'] == 'no').count()/df.count() * 100

88.30151954170445

### Give the maximum, mean, and minimum age of the average targeted customer

In [9]:
df.describe().select(['summary','age']).show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|             45211|
|   mean| 40.93621021432837|
| stddev|10.618762040975405|
|    min|                18|
|    max|                95|
+-------+------------------+



### Check the quality of customers by checking average balance, median balance of customers

In [10]:
df.registerTempTable("df_table")
spark.sql("SELECT AVG(BALANCE) as avg_balance, percentile_approx(BALANCE,0.5) as median_balance FROM df_table").show()

+------------------+--------------+
|       avg_balance|median_balance|
+------------------+--------------+
|1362.2720576850766|           448|
+------------------+--------------+



### Check if age matters in marketing subscription for deposit
ans: Age is **not** the factor as median and stddev is not having much difference

In [11]:
df.filter(df['y']=='yes').describe().select(['summary','age']).show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|              5289|
|   mean|41.670069956513515|
| stddev|13.497781462199919|
|    min|                18|
|    max|                95|
+-------+------------------+



In [12]:
df.filter(df['y']=='no').describe().select(['summary','age']).show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|             39922|
|   mean| 40.83898602274435|
| stddev|10.172662094342575|
|    min|                18|
|    max|                95|
+-------+------------------+



### Check if marital status mattered for a subscription to deposit
ans: `Single` people are having the highest persent(~15%) of enrollment among the all the categoreies but there id **no** storng relationship here as for all the marital categories succesful enrolment is in between 10%-15% which is not a big gap.

In [13]:
df.groupby("marital").count().registerTempTable("df_marital_total")
df.groupby("marital").count().show()

+--------+-----+
| marital|count|
+--------+-----+
|divorced| 5207|
| married|27214|
|  single|12790|
+--------+-----+



In [14]:
df.filter(df['y']=='yes').groupby("marital").count().registerTempTable("df_marital_yes")
df.filter(df['y']=='yes').groupby("marital").count().show()

+--------+-----+
| marital|count|
+--------+-----+
|divorced|  622|
| married| 2755|
|  single| 1912|
+--------+-----+



In [15]:
spark.sql("select t.marital, y.count/t.count from df_marital_total t inner join df_marital_yes y on t.marital=y.marital").show()

+--------+-----------------------------------------------+
| marital|(CAST(count AS DOUBLE) / CAST(count AS DOUBLE))|
+--------+-----------------------------------------------+
|divorced|                            0.11945458037257538|
| married|                            0.10123465863158668|
|  single|                             0.1494917904612979|
+--------+-----------------------------------------------+



### Check if age and marital status together mattered for a subscription to deposit scheme
ans: **Yes**, age and marital status together mattered for a subscription to deposit scheme

In [16]:
df.groupby(["marital","age"]).count().registerTempTable("df_marital_age_total")
df.groupby(["marital","age"]).count().show()

+--------+---+-----+
| marital|age|count|
+--------+---+-----+
| married| 56|  564|
|divorced| 77|    5|
| married| 47|  826|
|  single| 45|  146|
| married| 80|   29|
| married| 55|  589|
|divorced| 89|    1|
| married| 39|  960|
| married| 89|    2|
| married| 38|  905|
|divorced| 26|   20|
|  single| 73|    2|
| married| 33| 1075|
| married| 32| 1007|
|divorced| 95|    1|
|  single| 30| 1012|
| married| 22|    9|
|divorced| 84|    3|
|  single| 38|  394|
|divorced| 78|   12|
+--------+---+-----+
only showing top 20 rows



In [17]:
df.filter(df['y']=='yes').groupby(["marital","age"]).count().registerTempTable("df_marital_age_yes")
df.filter(df['y']=='yes').groupby(["marital","age"]).count().show()

+--------+---+-----+
| marital|age|count|
+--------+---+-----+
| married| 56|   49|
|divorced| 77|    3|
| married| 47|   83|
|  single| 45|   14|
| married| 80|   11|
| married| 55|   50|
| married| 39|   87|
| married| 38|   86|
|  single| 73|    1|
| married| 32|   87|
| married| 33|   97|
|divorced| 95|    1|
|  single| 30|  151|
|divorced| 84|    1|
|  single| 38|   38|
|divorced| 78|    6|
| married| 28|   20|
| married| 93|    2|
|  single| 20|   14|
| married| 26|   13|
+--------+---+-----+
only showing top 20 rows



In [19]:
spark.sql("select t.marital, t.age,y.count,t.count, y.count/t.count as s_rate from df_marital_age_total t inner join df_marital_age_yes y on t.marital=y.marital and t.age=y.age order by s_rate desc").show()

+--------+---+-----+-----+------------------+
| marital|age|count|count|            s_rate|
+--------+---+-----+-----+------------------+
| married| 92|    2|    2|               1.0|
|divorced| 95|    1|    1|               1.0|
|divorced| 90|    2|    2|               1.0|
|divorced| 68|    6|    6|               1.0|
|divorced| 87|    1|    1|               1.0|
| married| 93|    2|    2|               1.0|
|divorced| 85|    1|    1|               1.0|
|  single| 86|    1|    1|               1.0|
|divorced| 67|    7|    8|             0.875|
|divorced| 62|    5|    6|0.8333333333333334|
|divorced| 76|    6|    8|              0.75|
| married| 85|    3|    4|              0.75|
|divorced| 71|    8|   11|0.7272727272727273|
| married| 84|    4|    6|0.6666666666666666|
|divorced| 73|    4|    6|0.6666666666666666|
| married| 87|    2|    3|0.6666666666666666|
|divorced| 77|    3|    5|               0.6|
|  single| 18|    7|   12|0.5833333333333334|
|divorced| 63|    4|    7|0.571428

### Do feature engineering for the bank and find the right age effect on the campaign.
We need to do the bucketing of age (18-20,21-30...81-90,91-100) and then run the same analysis as done above